## Extracting the intensity data for analysis in MTEX

In [ ]:
import pathlib
import re

# from tqdm.notebook import tqdm
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from scipy.signal import medfilt

In [ ]:
pwd

### Extracting intensity data

Use the `read_fit_results` function to get the results from the .fit files.

In [ ]:
def read_fit_results (experiment_number: int, path_to_fit_file: str, start: int, end: int, peak_label: list):
    """This function loops through refined '.fit' files produced using
    the python package Continuous-Peak-Fit and searches for the peak position, 
    intensity, half-width and Pseudo-Voigt weighting results. 
    
    :param experiment_number: input experiment number.
    :param path_to_fit_file: input path to the fit results.
    :param start: first file number.
    :param end: last file number.
    :param peak_label: lattice plane peak labels.
    
    :return: peak position, peak intensity, peak half-width and peak Pseudo-Voigt weighted fraction
    as nested dictionaries.
    """

    # define dictionaries to save results to
    peak_position = dict()
    peak_intensity = dict()
    peak_halfwidth = dict()
    peak_PV_weight = dict()
    
    for test_number in range (start,end+1):
        # input fit results file
        # input_path = f"example-results/Diamond_2021/{experiment_number:05d}-cpf-output/{experiment_number:05d}_summed_{test_number:05d}.fit"
        input_path = path_to_fit_file + f"{test_number:05d}.fit"
        
        # dictionary to save results to
        peak_position[test_number] = dict()
        peak_intensity[test_number] = dict()
        peak_halfwidth[test_number] = dict()
        peak_PV_weight[test_number] = dict()
        
        with open(input_path, 'r') as results_file:  
            line = results_file.readline()
            peak_number = 0
            
            while line:
                if '# peak number' in line:
                    peak_position[test_number][peak_label[peak_number]]=[]
                    peak_intensity[test_number][peak_label[peak_number]]=[]
                    peak_halfwidth[test_number][peak_label[peak_number]]=[]
                    peak_PV_weight[test_number][peak_label[peak_number]]=[]

                    for azimuth_degree in range (0, 360):
                        line = results_file.readline()
                        fit_result = re.findall(r"[-+]?\d*\.\d+|\d+", line)
                        peak_position[test_number][peak_label[peak_number]].append(float(fit_result[0]))
                        peak_intensity[test_number][peak_label[peak_number]].append(float(fit_result[1]))
                        peak_halfwidth[test_number][peak_label[peak_number]].append(float(fit_result[2]))
                        peak_PV_weight[test_number][peak_label[peak_number]].append(float(fit_result[3]))

                    peak_number+=1

                else:
                    line = results_file.readline()

    return peak_position, peak_intensity, peak_halfwidth, peak_PV_weight

#### Extracting experiment data

Be sure to correctly identify all peaks contained in the .fit data file.

In [ ]:
peak_label = ['10-10',
              '0002',
              '110',
              '10-11',
              '10-12',
              '200',
              '11-20',
              '10-13',
              #'211'
              '20-20',
              '11-22',
              '20-21',
              '0004',
              '220',
              '20-22',
              '10-14',
              '310',
              '20-23',
              '21-30',
              '21-31',
              #'222'
              '11-24',
              '21-32',
              '10-15',
              '20-24',
              #'321',
              '30-30',
              '30-31',
              #'21-33'
             ]

In [ ]:
experiment_number = 103851
# enter the input path excluding the test number
path_to_fit_file = f"example-results/Diamond_2021/{experiment_number:05d}-cpf-output/{experiment_number:05d}_summed_" 
start = 1
end = 1

peak_position, peak_intensity, peak_halfwidth, peak_PV_weight = read_fit_results(experiment_number,path_to_fit_file,start,end,peak_label)

#### Extracting calibration powder data

In [ ]:
powder_peak_label = ['10-10',
              '0002',
              #'110',
              '10-11',
              '10-12',
              #'200',
              '11-20',
              '10-13',
              #'211'
              '20-20',
              '11-22',
              '20-21',
              '0004',
              #'220',
              '20-22',
              '10-14',
              #'310',
              '20-23',
              '21-30',
              '21-31',
              #'222'
              '11-24',
              '21-32',
              '10-15',
              '20-24',
              #'321',
              '30-30',
              '30-31',
              #'21-33'
             ]

In [ ]:
powder_experiment_number = 103818
# enter the input path excluding the test number
path_to_powder_fit_file = f"example-results/Diamond_2021/{powder_experiment_number:05d}-cpf-output/{powder_experiment_number:05d}_summed_" 
start = 1
end = 1

powder_peak_position, powder_peak_intensity, powder_peak_halfwidth, powder_peak_PV_weight = read_fit_results(powder_experiment_number,path_to_powder_fit_file,start,end,powder_peak_label)

## Calibrating intensity data to powder measurements

In [ ]:
start = 1
end = 1

corrected_peak_intensity = dict()

for test_number in range (start,end+1):
    corrected_peak_intensity[test_number] = dict()
    
    for label in powder_peak_label:
        powder_average = np.average(powder_peak_intensity[test_number][label])
        powder_error = np.std(powder_peak_intensity[test_number][label], ddof=1)
        
        corrected_peak_intensity[test_number][label] = []                            
        corrected_peak_intensity[test_number][label] = peak_intensity[test_number][label] / powder_average
        
        print(f"Normalised {label} intensities with a value of {powder_average} +/- {powder_error} from average powder intensity.")

### Writing intensity data to file

In [ ]:
def peak_data_to_text_file(experiment_number: int, intensity_type: str, peak_intensity: dict, peak_label: list, 
                           orientation: str, data_resolution: int):
    """This function calculates the corresponding spherical polar coordinates
    (polar and azimuthal angles) for the set of intensity values and 
    writes out a text file of the data that can be read in MTEX.
    
    :param experiment_number: input experiment number.
    :param intensity_type: name output path of text file, either 'raw' or 'powder-corrected'.
    :param peak_intensity: diffraction ring intensities from different lattice planes.
    :param peak_label: lattice plane peak labels.
    :param orientation: sample name setting lattice plane orientation, 'S1': 'YZ', 'S2': 'XZ', 'S3': 'XY', 'S4': 'YX to YZ', 'S5': 'XY to XZ', 'S6': 'XZ to YZ'.
    :param data_resolution: resolution of intensities in degrees, usually set as 1 degree.
    
    """
    
    sample_to_plane = {'S1': 'YZ', 'S2': 'XZ', 'S3': 'XY', 'S4': 'YX to YZ', 'S5': 'XY to XZ', 'S6': 'XZ to YZ'}
    plane = sample_to_plane[orientation]
    
    for label in peak_label:
        output_path = f'example-results/Diamond_2021/{experiment_number:05d}-{intensity_type}-intensities/{experiment_number:05d}_peak_intensity_{label}.txt'
        output_folder = pathlib.Path(output_path).parent
        output_folder.mkdir(exist_ok=True)

        with open(output_path, 'w') as output_file:

            # write metadata for the top of the file
            output_file.write('Polar Angle \t Azimuth Angle \t Intensity \n')
            
            # iterate through the peak intensities
            for i in range(0, len(peak_intensity[1][label])):
                
                # set the angle around the circle (diffraction pattern ring) from horizontal position
                alpha = i*data_resolution
                
                # set the inclination of any inclined planes to 45 degrees
                beta = (np.pi)/4

                # use the circle parametrisation formula to calculate x,y,z values
                if plane == 'XY':
                    x = np.cos(np.deg2rad(alpha))
                    y = np.sin(np.deg2rad(alpha))
                    z = 0

                if plane == 'XZ':
                    x = np.cos(np.deg2rad(alpha))
                    y = 0
                    z = np.sin(np.deg2rad(alpha))

                if plane == 'YZ':
                    x = 0
                    y = np.cos(np.deg2rad(alpha))
                    z = np.sin(np.deg2rad(alpha))

                if plane == 'XY to XZ':
                    x = np.cos(np.deg2rad(alpha))
                    y = np.sin(np.deg2rad(alpha))*np.cos(beta)
                    z = np.sin(np.deg2rad(alpha))*np.sin(beta)

                if plane == 'YX to YZ':
                    x = np.sin(np.deg2rad(alpha))*np.cos(beta)
                    y = np.cos(np.deg2rad(alpha))
                    z = np.sin(np.deg2rad(alpha))*np.sin(beta)

                if plane == 'XZ to YZ':
                    x = np.cos(np.deg2rad(alpha))*np.cos(beta)
                    y = np.cos(np.deg2rad(alpha))*np.sin(beta)
                    z = np.sin(np.deg2rad(alpha))

                # calulate the spherical polar coordinates from cartesian coordinates using the standard equations
                theta = np.rad2deg(np.arccos(z))
                # arctan2 needed to handle negative values for phi values
                phi = np.rad2deg(np.arctan2(y,x))
                # convert phi scale from -180, 180 to 0, 360
                if phi < 0:
                    phi = phi + 360
                
                output_file.write(f'{theta}\t'
                                  f'{phi}\t'
                                  f'{peak_intensity[1][label][i]}\n')
                
        print(f"Written .txt data file to: '{output_path}'.")

In [ ]:
# intensity type, either 'raw' or 'powder-corrected'
intensity_type = 'raw'
# sample orientation from options S1, S2, S3, S4, S5, S6
orientation = 'S3'
# data resolution in degrees
data_resolution = 1

peak_data_to_text_file(experiment_number, intensity_type, peak_intensity, peak_label, orientation, data_resolution)

In [ ]:
# intensity type, either 'raw' or 'powder-corrected'
intensity_type = 'powder-corrected'

peak_data_to_text_file(experiment_number, intensity_type, corrected_peak_intensity, powder_peak_label, orientation, data_resolution)

### Notes on parameterising a circle in 3D and converting to spherical polar coordinates

The intensity data must be associated with the correct spherical polar coordinates, so that it can be easily loaded and analysed in `MTEX`.

**Diffraction pattern rings**

Each diffraction pattern ring is a 2D circle of intensity points collected on the detector, with the spots around the ring directly related to orientation of a grain's lattice plane within the sample *i.e. diffraction spots at the top or bottom of the ring correspond to grains with their lattice plane aligned parrallel to the top or bottom of the sample.* Therefore, This diffraction pattern ring can be thought of as the edge of a pole figure. In a synchrotron experiment we typically only measure a small subset of sample orientations, but to plot this data correctly we need to represent our 2D plane of intenstity measurements in 3D, to calculate the crystal structure with respect to the sample directions e.g. as an ODF or pole figure.

**Spherical polar coordinates**

To handle 3D intensity data for different lattice planes, MTEX uses spherical polar coordinates. The polar coordinates are defined by a polar angle, ${\theta}$, from ${0^\circ}$ (North) to ${180^\circ}$ (South), and an azimuthal angle, ${\phi}$, from ${0^\circ}$ to ${360^\circ}$. The X-axis is aligned along ${\phi = 0^\circ}$, the Y-axis is along ${\phi = 90^\circ}$ and the Z-axis is along ${\theta = 0^\circ}$. Note, this is using the *physics* convention of polar coordinates, rather than the *maths* convention, and there are no negative values in this definition. 

**Sample orientaion**

For our experiment there were six different rolled sample orientations. The typical convention is that ${X = RD}$, ${Y = TD}$ and ${Z = ND}$. For this demonstration and to avoid confusion we will use the alignments with respect to X, Y and Z. So, there are six possible planes our circle (of intensity points) could lie along in 3D;
    
| Sample Name | Horizontal-Vertical Alignment (Rolled Sample) | Horizontal-Vertical Alignment (Cartesian) |
| -- | --------- | --- |
| S3 | TD-RD     | X-Y |
| S2 | RD-ND     | X-Z |
| S1 | TD-ND     | Y-Z |
| S5 | RD-TD45ND | Tilt between X-Y and X-Z |
| S4 | TD-RD45ND | Tilt between Y-X and Y-Z |
| S6 | TD45RD-ND | Tilt between X-Z and Y-Z |

**Parameterising a circle**

To plot the path of a circle in 3D, we need to parameterise the solution. The parameteric solution for a circle lying in the X-Y plane, with a radius, R, and centred at (0,0) is ${X^2 + Y^2 = R^2}$, where ${X = R \cos(\alpha)}$ and ${Y = R \sin(\alpha)}$ and ${\alpha \in [0, 2 \pi]}$. If the circle lies on a different plane then we assign the corresponding X (horizonatal) and Y (vertical) axes.

Then, if our circle is tilted, we need to describe the tilt, as is shown [here](https://math.stackexchange.com/questions/215880/equation-for-making-a-circle-in-3d-space). Let's take the circle tilted between X-Y and X-Z as an example. 

First, we parameterise the expression for the circle on each plane, which gives us the solution ${X = R \cos(\alpha)}$, ${Y = R \sin(\alpha)}$ and ${Z = R \sin(\alpha)}$. 

The plane of the circle can lie at an angle ${\beta \in [0, \pi / 2]}$ from the horizontal, between the Y and Z axis. This defines a right-angled triangle which we can solve using trigonometry. So, the adjacent side is ${Y = \cos(\beta)}$ and the opposite side is ${Z = \sin(\beta)}$. 

Putting this together, then gives us our parameteric solution to draw a circle of points that is centred at (0,0), with ${\alpha \in [0, 2 \pi]}$, and titled between X-Y and X-Z by an angle ${\beta \in [0, \pi / 2]}$. In our case, ${\beta = \pi / 4 = 45 ^\circ}$, and ${R = 1}$;

${X = \cos(\alpha)}$ 

${Y = \sin(\alpha).\cos(\beta)}$

${Z = \sin(\alpha).\sin(\beta)}$

**Converting from *Cartesian* to *Spherical Polar* coordinates**

This plots the path of our circle in *cartesian coordinates*. However, we then need to convert this to *spherical polar coordinates* to match with our intensity data and to feed into MTEX.

To convert to polar coordinates we use the standard formula with the *physics* convention described [here]( https://en.wikipedia.org/wiki/Spherical_coordinate_system);

Polar Angle: ${\theta = \arccos(Z / R)}$

Azimuthal Angle: ${\phi = \arctan(Y / X)}$

However, we need to use the 2-argument arctangent ([atan2](https://en.wikipedia.org/wiki/Atan2)) expression to handle negative X and Y values. And, the above expression will return azimuthal angles within the range from -180 to 180, which will need to be converted to a range from 0 to 360 (which is the convention in MTEX).

In [ ]:
planes = 'XY', 'XZ', 'YZ', 'XY to XZ', 'YX to YZ', 'XZ to YZ'

x_values={}
y_values={}
z_values={}

polar={}
azimuth={}

resolution = 1
number_iterations = int((359 / resolution) + 1)

for plane in planes:
    x_values[plane]=[]
    y_values[plane]=[]
    z_values[plane]=[]
    polar[plane]=[]
    azimuth[plane]=[]
    
    # draw the circle from 0 to 360 degrees
    for alpha in np.linspace(0,359,number_iterations):
        
        # set the inclination of any inclined planes to 45 degrees
        beta = (np.pi)/4
        
        # use the circle parametrisation formula to calculate x,y,z values
        if plane == 'XY':
            x = np.cos(np.deg2rad(alpha))
            y = np.sin(np.deg2rad(alpha))
            z = 0

        if plane == 'XZ':
            x = np.cos(np.deg2rad(alpha))
            y = 0
            z = np.sin(np.deg2rad(alpha))

        if plane == 'YZ':
            x = 0
            y = np.cos(np.deg2rad(alpha))
            z = np.sin(np.deg2rad(alpha))
            
        if plane == 'XY to XZ':
            x = np.cos(np.deg2rad(alpha))
            y = np.sin(np.deg2rad(alpha))*np.cos(beta)
            z = np.sin(np.deg2rad(alpha))*np.sin(beta)

        if plane == 'YX to YZ':
            x = np.sin(np.deg2rad(alpha))*np.cos(beta)
            y = np.cos(np.deg2rad(alpha))
            z = np.sin(np.deg2rad(alpha))*np.sin(beta)

        if plane == 'XZ to YZ':
            x = np.cos(np.deg2rad(alpha))*np.cos(beta)
            y = np.cos(np.deg2rad(alpha))*np.sin(beta)
            z = np.sin(np.deg2rad(alpha))
        
        # calulate the spherical polar coordinates from cartesian coordinates using the standard equations
        theta = np.rad2deg(np.arccos(z))
        # arctan2 needed to handle negative values for phi values
        phi = np.rad2deg(np.arctan2(y,x))
        # convert phi scale from -180, 180 to 0, 360
        if phi < 0:
            phi = phi + 360

        # store the data in dictionaries
        x_values[plane].append(x)
        y_values[plane].append(y)
        z_values[plane].append(z)
        polar[plane].append(theta)
        azimuth[plane].append(phi)

Note, the `XZ` and `XZ to YZ` circles do not seem to render correctly on the plot. But, if you check the values you will see they are orientated correctly.

In [ ]:
fig = plt.figure(figsize = (10,10))
ax = plt.axes(projection='3d')
for plane in planes:
    ax.plot3D(x_values[plane], y_values[plane], z_values[plane], label=plane)
    ax.set_xlabel('x')
    ax.set_ylabel('y')
    ax.set_zlabel('z')
    ax.legend(loc ='upper left')

In [ ]:
count=1
plt.subplots(figsize=(15, 15))

for plane in planes:    
    plt.subplot(6,2,count)
    plt.plot(x_values[plane], color = 'purple', label = r'x-axis')
    plt.plot(y_values[plane], color = 'green', label = r'y-axis')
    plt.plot(z_values[plane], color = 'orange', label = r'z-axis')
    plt.xlabel(r'Rotation Angle, ${\alpha}$ (${^\circ}$)')
    plt.ylabel(r'Position along axis')
    plt.title(plane)
    plt.legend()
    plt.tight_layout()
    count+=1
                
    plt.subplot(6,2,count)
    plt.plot(polar[plane], color = 'red', label = r'Polar, ${\theta}$')
    plt.plot(azimuth[plane], color ='blue', label = r'Azimuthal Angle, ${\phi}$')
    plt.title(plane)
    plt.xlabel(r'Count')
    plt.ylabel(r'Spherical Polar Coordinates (${^\circ}$)')
    plt.legend()
    plt.tight_layout()       
    count+=1            
                
plt.show()

### Combine intensities from different samples

In [ ]:
def combine_files (experiment_number: list, intensity_type: str, peak_label: list):
    """This function combines the intensity and spherical polar coordinates of
    multiple files into one file, by writing the data sequentially to a new 
    text file.
    
    :param experiment_number: input experiment number.
    :param intensity_type: name input and output path of text file, either 'raw' or 'powder-corrected'.
    :param peak_label: lattice plane peak labels.
    
    """
    
    start = experiment_number[0]
    end = experiment_number[-1]

    for label in peak_label:
        output_path = f"example-results/Diamond_2021/{start}-to-{end}-combined-{intensity_type}-intensities/combined_peak_intensity_{label}.txt"
        output_folder = pathlib.Path(output_path).parent
        output_folder.mkdir(exist_ok=True)
        # overwrite any existing output file
        
        with open(output_path, 'w') as output_file:
            # write metadata for the top of the file
            output_file.write('Polar Angle \t Azimuth Angle \t Intensity \n')
        
        for experiment in experiment_number:
            input_path = f"example-results/Diamond_2021/{experiment:05d}-{intensity_type}-intensities/{experiment:05d}_peak_intensity_{label}.txt"
       
            # open output file in append mode to add lines
            with open(output_path, 'a') as output_file:
                
                with open(input_path, 'r') as results_file:  
                    line = results_file.readline()
                    line = results_file.readline()
                    
                    while line:
                        output_file.write(line)
                        line = results_file.readline()

                    print(f"Written data from: '...{input_path[47:]}' to: '...{output_path[66:]}'.")    

In [ ]:
intensity_type = 'raw'

experiment_number = list(range(103840,103846))
print(experiment_number)

combine_files(experiment_number, intensity_type, peak_label)

In [ ]:
intensity_type = 'powder-corrected'

combine_files(experiment_number, intensity_type, powder_peak_label)